# Título   Crear clase sample

## Autor: Jose Chelquer
## Fecha de última modificación: 24/10/2025
## Descripción:

Crear un sample del dataset con proporciones parametrizadas y meses seleccionados

## Parámetros

< Descripción de cada uno de los parámetros que utiliza el job >


In [1]:

usar_gdrive=False    # Poner en true si se va a ejecutar con archivos de google drive

feature_a_usar = 'clase_ternaria'     #feature a la que hacen referencia los valores de % a seleccionar
sample_pct={'BAJA+1':1, 'BAJA+2':1, 'CONTINUA':.05}   # % de cada clase a obtener en el sample
# meses_a_seleccionar = [201908, 201909, 201910]      # Versión para seleccionar meses específicos
meses_a_seleccionar=[]                                # Versión para seleccionar todos los meses

## Input

< Archivos de datos (csv.gz) con sus paths que van a consumirse por el job>

In [2]:
# Ejemplo


dataset_path = '~/buckets/b1/datasets' 
dataset_file = 'competencia_02.csv.gz'

## Output

< Archivos, bases de datos, modelos que va a generar el job>

In [3]:
output_file='competencia_02_sampleado.csv.gz'


In [4]:
## Procesos

### Paquetes necesarios

## Código del proceso

< Todo el código a partir de aquí debe poder ejecutarse sin necesidad de parametrizar nada>

In [5]:
import os
import pandas as pd

In [6]:
if usar_gdrive:
    print ('Cargando google drive')
    from google.colab import drive
    drive.mount('/content/drive')

In [ ]:
input_gz=dataset_file.endswith('.gz')
output_gz=output_file.endswith('.gz')

In [7]:
# leer el dataset

file_path = os.path.join(dataset_path, dataset_file)
print (f'leyendo {file_path}')
# Read the CSV file into a DataFrame
df = pd.read_csv(file_path)

print (f'Se leyeron {df.shape[0]} filas')

leyendo ~/buckets/b1/datasets/competencia_02.csv.gz
Se leyeron 4735593 filas


In [8]:
# Seleccionar meses
if meses_a_seleccionar:
    print (f'seleccionando meses {meses_a_seleccionar}')
    df=df[df['foto_mes'].isin(meses_a_seleccionar)]
    print (f'quedaron {df.shape[0]} filas')

In [9]:
# muestrar
# Función para filtrar y muestrear por cada valor del feature
def filtrar_muestrear(df, feature, sample_pct):
    df_sampled = pd.concat([
        df[df[feature] == clase].sample(frac=sample_pct[clase], random_state=42)
        for clase in sample_pct
    ])
    return df_sampled

# Aplicamos la función al DataFrame
df = filtrar_muestrear(df, feature_a_usar, sample_pct)

print (f'quedaron {df.shape[0]} filas')


quedaron 588402 filas


In [ ]:
output_file_path=os.path.join(dataset_path, output_file)
if output_gz:
    df.to_csv('nombre_archivo.csv.gz', index=False, compression='gzip')
else:
    df.to_csv('nombre_archivo.csv.gz', index=False)
    